In [ ]:
import json
import pandas as pd
import os

# 讀取 JSON 檔案
file_path = '/Users/chianlee/Desktop/disney/data/future_land_2024-09-01.json'

# 從檔案名稱中提取日期部分
file_name = os.path.basename(file_path)
selected_date = file_name.split('_')[-1].split('.')[0]  # 提取日期 "2024-09-01"

# 打開並讀取 JSON 檔案
with open(file_path, 'r') as file:
    data = json.load(file)

# 將資料轉換為 DataFrame
df = pd.DataFrame(data)

# 選擇指定日期
df_selected_date = df[df['Date'] == selected_date]

# 計算每個設施等待時間最短的前三個時段
top_3_shortest_times = df_selected_date.groupby('FacilityEnglish').apply(lambda x: x.nsmallest(3, 'PredictedWaitTime')).reset_index(drop=True)

# 計算每個設施等待時間最長的前三個時段
top_3_longest_times = df_selected_date.groupby('FacilityEnglish').apply(lambda x: x.nlargest(3, 'PredictedWaitTime')).reset_index(drop=True)

# 將結果轉換為 JSON 格式
result = {
    'shortest_times': top_3_shortest_times.groupby('FacilityEnglish').apply(lambda x: x[['Hour', 'PredictedWaitTime']].to_dict(orient='records')).to_dict(),
    'longest_times': top_3_longest_times.groupby('FacilityEnglish').apply(lambda x: x[['Hour', 'PredictedWaitTime']].to_dict(orient='records')).to_dict()
}

# 保存為 JSON 檔案
output_file = f'/Users/chianlee/Desktop/disney/data/recom_time_land_{selected_date}.json'

with open(output_file, 'w') as json_file:
    json.dump(result, json_file, indent=4)

print(f"最短和最長等待時間的前三個時段已保存至 {output_file}")


In [ ]:
import json
import pandas as pd

# 讀取 JSON 檔案
file_path = '/Users/chianlee/Desktop/disney/data/future_land_2024-08-10_to_2024-08-31.json'

with open(file_path, 'r') as file:
    data = json.load(file)

# 將資料轉換為 DataFrame
df = pd.DataFrame(data)

# 選擇指定日期（例如 "2024-08-10"）
selected_date = "2024-08-10"
df_selected_date = df[df['Date'] == selected_date]

# 計算每個設施等待時間最短的前三個時段
top_3_shortest_times = df_selected_date.groupby('FacilityEnglish').apply(lambda x: x.nsmallest(3, 'PredictedWaitTime')).reset_index(drop=True)

# 計算每個設施等待時間最長的前三個時段
top_3_longest_times = df_selected_date.groupby('FacilityEnglish').apply(lambda x: x.nlargest(3, 'PredictedWaitTime')).reset_index(drop=True)

# 將結果轉換為以設施為單位的 JSON 格式
result = {}
facilities = df_selected_date['FacilityEnglish'].unique()

for facility in facilities:
    result[facility] = {
        'shortest_times': top_3_shortest_times[top_3_shortest_times['FacilityEnglish'] == facility][['Hour', 'PredictedWaitTime']].to_dict(orient='records'),
        'longest_times': top_3_longest_times[top_3_longest_times['FacilityEnglish'] == facility][['Hour', 'PredictedWaitTime']].to_dict(orient='records')
    }

# 保存為 JSON 檔案
output_file = f'/Users/chianlee/Desktop/disney/data/recom_time_land_{selected_date}.json'

with open(output_file, 'w') as json_file:
    json.dump(result, json_file, indent=4)

print(f"每個設施的最佳與最糟時段已保存至 {output_file}")